<a href="https://colab.research.google.com/github/profteachkids/CHE2064_Fall2021/blob/main/AdiabaticFlameTemperature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone --depth 1 https://github.com/profteachkids/chetools &> /dev/null
import sys
sys.path.insert(1, "/content/chetools") #Path to CHE module imports

In [ ]:
import numpy as np
from scipy.optimize import root
import tools.che as che
R=8.314 # J/(mol K)

In [17]:
p = che.Props(['Methane', 'Ethane', 'nPropane', 'Oxygen', 'Nitrogen', 'CarbonDioxide', 'Water'])

In [ ]:
def deltaHsensIG(T):
        return (p.CpIGA*T + p.CpIGB * p.CpIGC/np.tanh(p.CpIGC/T) - p.CpIGD * p.CpIGE * np.tanh(p.CpIGE/T))/1000

In [18]:
np.dot(np.array([1.,1., 1., 1., 2., 2., 3.,]), p.HfIG+ deltaHsensIG(400) - deltaHsensIG(298.15))

-1730339.765306445

In [19]:
p.HfIG+ deltaHsensIG(400) - deltaHsensIG(298.15)

array([ -70980.61819642,  -77780.56803685,  -96055.48335928,
          3022.12507945,    2971.49034285, -389689.10167981,
       -238369.99937314])

In [ ]:
# 2*(79/21) N2 + 2O2 + CH4 -> 2*(79/21)N2 + CO2 + 2H2O
# (7/2)*(79/21) N2 + (7/2)O2 + C2H6 -> (7/2)*(79/21)N2 + 2CO2 + 3H2O

In [44]:
excess = 0.
feedT = 300.
fuelCHFormula = np.array([[1,4],
                   [2,6],
                   [3,8]])
nfuel = np.array([1.,0.,0.])
nC, nH = nfuel @ fuelCHFormula
nCO2 = nC
nH2O = nH/2
nO2 = (nC + nH2O/2)*(1+excess)
nN2 = 79/21*nO2

feedn = np.concatenate((nfuel, np.array([nO2, nN2, 0., 0.])))
prodn = np.concatenate((np.zeros_like(nfuel), np.array([(nC+nH2O/2)*excess, nN2, nCO2, nH2O])))

In [49]:
def deltaHsensIG(T):
    return (p.CpIGA*T + p.CpIGB * p.CpIGC/np.tanh(p.CpIGC/T) - p.CpIGD * p.CpIGE * np.tanh(p.CpIGE/T))/1000

def deltaHIG(n,T):
    return np.dot(n, p.HfIG + deltaHsensIG(T))


In [57]:
deltaHIG(prodn, 2300.)

270271.9537297792

In [52]:
deltaHIG(feedn,300.)

283997.2265262645

In [58]:
def energy_balance(T):
    return deltaHIG(prodn, T) - deltaHIG(feedn,feedT)

In [59]:
root(energy_balance, [2000.])

    fjac: array([[-1.]])
     fun: 4.0745362639427185e-10
 message: 'The solution converged.'
    nfev: 6
     qtf: array([0.00077585])
       r: array([-441.0868811])
  status: 1
 success: True
       x: array([2331.13852626])